In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

# Hilfsfunktionen für die Spielimplementierung

In dieser Dateien werden Hilfsfunktionien implementiert, die für die grundlegende Spielimplementierung benötigt werden.

## Hilfsfunktionen für Spielsteine

Nachfolgend werden alle Hilfsfunktionen implementiert, die für das Interagieren mit den Spielsteinen benötigt werden.

Die Funktion `hasPlaceableStones` überprüft, ob ein Spieler für einen Zustand noch zusetzende Steine auf dem Stapel (_stash_) besitzt. Die Funktion hat zwei Argumente:

- `s` $\in States$;
- `p` $\in Player$.

Die Funktion gibt ein booleschen Wert zurück.

In [ ]:
def hasPlaceableStones(s, p):
    ((cw, cb), _) = s
    return cw >= 1 if p == 'w' else cb >= 1

Die Funktion `countStones` zählt die Steine eines Spieler auf einem Spielbrett. Die Funktion hat zwei Argumente:

- `s` $\in States$;
- `p` $\in Player$.

Die Funktion zählt nur die Steine auf dem Brett, nicht die Steine auf dem Stapel und gibt diese als Ganzzahl zurück.

In [ ]:
def countStones(s, p):
    (_, board) = s
    return [cell for ring in board for cell in ring].count(p)

Die Funktion `isAllowedToJump` überprüft, ob ein Spieler bei einem gegebenen Zustand seine Steine beliebig bewegen darf. Die Funktion hat zwei Argumente:

- `s` $\in States$;
- `p` $\in Player$.

Ein Spieler darf mit seinen Steinen springen, g.d.w. er weniger als drei Steine auf dem Spielbrett hat und sich keine Steine mehr von dem Spieler auf dem Stapel befinden. Die zweite Bedingung wird nicht überprüft, weil davon ausgegangen wird, dass die Funktion `hasPlaceableStones` zuvor aufgerufen wird.

Die Funktion gibt einen booleschen Wert zurück. 

In [ ]:
def isAllowedToJump(s, p):
    return countStones(s, p) <= 3

Die Funktion `hasEnoughStones` überprüft, ob ein Spieler noch genügend Steine zum Weiterspielen übrig hat. Die Funktion hat zwei Argumente:

- `s` $\in States$;
- `p` $\in Player$.

Ein Spieler hat genau dann genügend Steine, wenn er noch Steine zum Setzen auf dem Stapel hat oder er mindestes drei Steine auf dem Spielbrett besitzt.

Die Funktion gibt einen booleschen Wert zurück.

In [ ]:
def hasEnoughStones(s, p):
    return hasPlaceableStones(s, p) or countStones(s, p) >= 3

Die Funktion `removeFromStash` entfernt einen Stein von dem Stapel des gegebenen Spielers. Die Funktion hat zwei Argumente:

- `stash` ist ein Stapel;
- `p` $\in Player$.

Die Funktion gibt den neuen Stapel zurück.

In [ ]:
def removeFromStash(stash, p):
    return (stash[0] - (1 if p == 'w' else 0), stash[1] - (1 if p == 'b' else 0))

## Hilfsfunktionen für Spieler

In diesem Kapitel werden Hilfsfunktionen für die Spieler implementiert.

Die Funktion `opponent` nimmt einen Spieler und gibt den Gegenspieler zurück. Die Funktion hat ein Argument:

- `p` $\in Player$.

Da Mühle ein Zwei-Personen-Spiel ist, gibt es für die Funktion nur zwei Fälle:

- bei dem weißen Spieler `'w'` wird der Gegenspieler schwarz `'b'` zurückgegeben,
- ansonsten wird standardmäßig weiß `'w'` als Gegenspieler zurückgegeben.

In [ ]:
def opponent(p):
    return 'b' if p == 'w' else 'w'

Die Funktion `getPlayerAt` gibt den Spieler an der gegeben Koordinate des Spielbrettes zurück. Die Funktion hat zwei Argumente:

- `board` ist ein Spielbrett;
- `coord` ist eine Koordinate, die überprüft werden soll. 

Die Funktion gibt einen Spieler zurück. Falls an dieser Koordinate sich kein Spielerstein befinden sollte, wird entsprechend `' '` zurückgegeben.

In [ ]:
def getPlayerAt(board, coord):
    (r, c) = coord
    return board[r][c]

Die Funktion `playerPhase` berechnet für einen gegebenen Zustand und einen Spieler die aktuelle Phase des Spielers. Die Funktion hat zwei Argumente:

- `s` $\in States$;
- `p` $\in Player$.

Die Funktion überprüft mit den beiden Hilfsfunktionen `hasPlaceableStones` und `isAllowedToJump` die Spielerphase und gibt diese als Ganzzahl zurück:

- `1` für die _placing_ Phase, g.d.w. der Spieler noch Steine auf dem Stapel hat (`hasPlaceableStones`);
- `2` für die _moving_ Phase, g.d.w. der Spieler nicht in Phase 1 oder 3 ist;
- `3` für die _flying_ Phase, g.d.w. der Spieler springen darf (`isAllowedToJump`).

In [ ]:
def playerPhase(s, p):
    if hasPlaceableStones(s, p):
        return 1
    elif isAllowedToJump(s, p):
        return 3
    else:
        return 2

## Hilfsfunktionen für Zellen

In diesem Kapitel werden Hilfsfunktionen für die Veränderung oder Untersuchung von Zelleneigenschaften implementiert.

Die Funktion `findCellsOf` sucht auf einem Spielbrett `board` nach allen Zellen, welche durch Steine des Spielers `p` belegt sind. Die Funktion hat folgende Argumente:

- `board` ist ein Spielbrett;
- `p` $\in Player$.

Zurückgegeben wird die Menge aller Zellen wo gilt `board[r][c] = p`.

In [ ]:
def findCellsOf(board, p):
    return {(r, c) for r in range(0, 3) for c in range(0, 8) if board[r][c] == p}

Die Funktion `findEmptyCells` sucht auf dem Spielfeld `board` nach allen leeren Zellen. Die Funkion nimmt lediglich ein Argument:

* `board` ist ein Spielbrett.

Die Funktion ruft die Funktion `findCellsOf` mit `' '` als Spieler auf und gibt das erhaltene Ergebnis zurück. Somit ist jeder erhaltenen Zelle kein Spieler `'b'` oder `'w'` zugeordnet.

In [ ]:
def findEmptyCells(board):
    return findCellsOf(board, ' ')

Die Funktion `findNeighboringEmptyCells` sucht für eine gegebene Zelle `coordinates` die freien Nachbarzellen. Die Funktion hat zwei Argumente:

* `board` ist ein Spielbrett;
* `coordinates` ist ein Tupel mit den Koordinaten der Ausgangszelle.

Die Funktion überprüft, welche der angrenzenden Zellen in der Menge von `findEmptyCells` enthalten ist und gibt diese zurück.

In [ ]:
def findNeighboringEmptyCells(board, coordinates):
    (rootR, rootC) = coordinates
    return {(r, c) for (r, c) in findEmptyCells(board) if (r == rootR and (c == (rootC + 7) % 8 or c == (rootC + 1) % 8)) 
                                                        or (c % 2 == 1 and c == rootC and (r == rootR - 1 or r == rootR + 1)) }


Die Funktion `place` platziert den Stein eines Spielers `player` auf dem Spielbrett `board`. Folgende Argumente benötigt die Funktion:

* `board` ist ein Spielbrett;
* `coordinates` ist ein Tupel mit den Koordinaten des zu platzierenden Steins;
* `player` $\in Player$.

Die Funktion platziert den Stein an den angegebenen Koordinaten und gibt das neue Spielbrett zurück.

In [ ]:
def place(board, coordinates, player):
    (r, c) = coordinates
    return tuple(
        tuple(
            player if (c == ic) and (r == ir) else board[ir][ic]
            for ic in range(0, 8)
        ) for ir in range(0, 3)
    )

Die Funktion `move` bewegt einen bereits plazierten Stein auf dem Spielbrett `board`. Die Funktion erhält drei Argumente:

* `board` ist ein Spielbrett;
* `src` ist ein Tupel mit den Koordinaten der Ausgangszelle;
* `des` ist ein Tupel mit den Koordinaten der Zielzelle.

Die Funktion platziert den Wert der Ausgangszelle in die Zielzelle und umgegekehrt und gibt das neue Spielbrett zurück.

In [ ]:
def move(board, src, des):
    src_r, src_c = src
    des_r, des_c = des
    content_src = board[src_r][src_c]
    content_des = board[des_r][des_c]
    return tuple(
        tuple(
            content_des if (r,c) == src else (content_src if (r,c) == des else board[r][c])
            for c in range(0, 8)
        ) for r in range(0, 3)
    )

## Hilfsfunktionen für Mühlen
In diesem Kapitel werden Hilfsfunktionen implementiert, die für Mühlen nützlich sind.

Die Funktion `findMills` berechnet für ein Spielbrett und einen Spieler alle Mühlen, die dieser Spieler aktuell hat. Die Funktion hat zwei Argumente:

* `board` ist ein Spielbrett;
* `p` $\in Players$.

Der Rückgabewert der Funktion ist eine Menge von Mühlen. Eine Mühle wird dabei als Menge (bzw. `frozenset`) dargestellt, die die Koordinaten der Steine als Tupel beinhaltet.

* Zunächst werden alle Mühlen entlang der Ringe ermittelt. Dazu werden, beginnend an jeder Ecke, die nächsten drei Steine überprüft, ob sie dem Spieler `p` gehören. Ist dies der Fall, werden alle Coordinaten in einer Menge (bzw. `frozenset`) gespeichert.
* Für die Mühlen entlang der Verbindungslinien zwischen den Ringen, wird das gleiche Prinzip verwendet. Hier wird über alle Steine in der Mitte einer Seite iteriert und entlang der Ringe überprüft.

In [ ]:
def findMills(board, p):
    return {
        mill
        for mill in {
            # Calculate all mills on the rings
            frozenset((r, (c+o)%8) for o in range(0, 3))
            for r in range(0, 3)
            for c in range(0, 8, 2)
        } | {
            # Calculate all mills crossing the rings
            frozenset((r, c) for r in range(0, 3))
            for c in range(1, 8, 2)
        }
        if all(
            board[r][c] == p
            for (r, c) in mill
        )
    }

Die Funktion `countMills` berechnet für ein Spielbrett, eine Menge bereits existierender Mühlen und einen Spieler die Anzahl der Mühlen, die neu entstanden sind. Die Funktion hat 3 Parameter:

* `board` ist ein Spielbrett;
* `oldMills` ist eine Menge von Mühlen (Menge von Koordinaten);
* `p` $\in Players$.


Der Rückgabewert dieser Funktion ist eine Ganzzahl für die gilt $return \in \{0,1,2\}$. Indem die Menge der bereits existierenden Mühlen von der Menge der aktuellen Mühlen abgezogen wird, erhält man die Menge der aktuellen Mühlen. Von diesen kann die Anzahl berechnet werden.

In [ ]:
def countNewMills(board, oldMills, p):
    return len(findMills(board, p) - oldMills)

Die Funktion `getCellsPoundable` berechnet für ein Spielbrett und einen Spieler, welche Steine des Gegeners dieser schlagen darf. Ein Stein darf geschlagen werden, wenn er sich nicht in einer Mühle befindet. Wenn sich alle Steine in einer Mühle befinden, dürfen alle Steine geschlagen werden. Die Funktion hat 2 Parameter:

* `board` ist ein Spielbrett;
* `p` $\in Players$.

Der Rückgabewert ist eine Menge von Koordinaten. Die Menge aller Koordinaten der Steine des Gegeners, sowie die Menge aller Koordinaten aller Steine aus den Mühlen des Gegeners werden errechnet und voneinander abgezogen. Falls diese Menge nicht leer ist, dürfen nur an diesen Positionen Steine geschlagen werden, ansonsten dürfen alle Steine des Gegners geschlagen werden.

In [ ]:
def getCellsPoundable(board, p):
    opponentCells = findCellsOf(board, opponent(p))
    
    opponentCellsInMills = {
        cell
        for mill in findMills(board, opponent(p))
        for cell in mill
    }
    
    return opponentCells - opponentCellsInMills if len(opponentCells - opponentCellsInMills) > 0 else opponentCells

Die Funktion `poundMills` errechnet aus einem Spielbrett, einer Anzahl an zu schlagenden Steinen und einem Spieler alle möglichen Spielbrettkonfigurationen, bei denen der Spieler die gegebene Anzahl an Steinen geschlagen hat. Die Funktion hat 3 Parameter:

* `board` ist ein Spielbrett;
* `count` $\in \{0, 1, 2\}$;
* `p` $\in Players$.

Der Rückgabewert ist eine Menge an Spielbrettern. Die Funktion ist rekursiv implementiert. Wenn kein Stein mehr geschlagen werden muss, wird eine Menge mit dem aktuellen Spielbrett zurück gegeben. Ansonsten wird rekursiv die Menge aller möglichen Spielbrettkonfigurationen mit einem zu schlagenden Stein weniger errechnet. Aus allen diesen Spielbrettern wird ein weiterer Stein entfernt, der geschlagen werden darf.

In [ ]:
def poundMills(board, count, p):
    if count <= 0:
        return { board }

    return {
        place(b, cell, ' ')
        for b in poundMills(board, count-1, p)
        for cell in getCellsPoundable(b, p)
    }